# Data processing

This code import the accelerations of the segmented gait cycles and computes the correlation matrixes

In [1]:
import numpy as np
import numpy.ma as ma
import h5py
import scipy.io
import matplotlib.pyplot as plt
import os

In [ ]:
def corr_coefs(x, y):
    """
    Compute Pearson correlation coefficient between two arrays.
    
    Parameters
    ----------
    x : array-like
        First input array
    y : array-like
        Second input array
    
    Returns
    -------
    float or ndarray
        Pearson correlation coefficient(s) between x and y.
        Uses masked array operations to handle missing data.
    
    Notes
    -----
    This function computes the correlation using the formula:
    r = sum((x - mean(x)) * (y - mean(y))) / sqrt(sum((x - mean(x))^2) * sum((y - mean(y))^2))
    
    Uses numpy.ma (masked array) functions to handle potential masked/missing values.
    """
    return ma.sum((x - x.mean(axis=0)) * (y - y.mean(axis=0)), axis=0) / ma.sqrt(
        ma.sum((x - x.mean(axis=0)) ** 2, axis=0) * ma.sum((y - y.mean(axis=0)) ** 2, axis=0)
    )

In [2]:
def get_abs_C(Xm):
    """
    Compute pairwise correlation matrix for markers across dimensions.
    
    Parameters
    ----------
    Xm : array-like, shape (n_markers, n_timepoints, 3)
        Marker position data with 3 spatial dimensions
    
    Returns
    -------
    C : ndarray, shape (n_markers, n_markers, 3)
        Correlation coefficient matrix for each dimension.
        C[i,j,k] contains correlation between marker i and j in dimension k.
        Diagonal elements are set to 1.
    
    Notes
    -----
    This function computes correlations symmetrically and fills both
    upper and lower triangular portions of the matrix.
    """
    # Get number of markers
    n_markers = Xm.shape[0]
    
    # Initialize correlation tensor
    C = np.zeros((n_markers, n_markers, 3))
    
    # Compute pairwise correlations
    for i in range(n_markers):
        for j in range(i + 1, n_markers):
            x = Xm[i]
            y = Xm[j]
            for kd in range(3):
                c_ = corr_coefs(x[:, kd], y[:, kd])
                C[i, j, kd] = c_
                C[j, i, kd] = c_
    
    # Set diagonal to 1 (self-correlation)
    C[np.arange(n_markers), np.arange(n_markers), :] = 1
    
    return C

In [3]:
f = h5py.File("raw_gait_cycle_kinematics.h5",'r')
fg = f['data']
labels = list(f['data'].keys())
accel_subj_cond=[]
vel_subj_cond=[]
X_subj_cond=[]
labels_subj_cond=[]
for k in range(len(labels)):
    fl = fg[labels[k]]
    trials = list(fl.keys())
    a_trials=[]
    v_trials=[]
    X_trials=[]
    for i in range(len(trials)):
        a = np.array(fl[trials[i]]['accel'])
        v = np.array(fl[trials[i]]['vel'])
        X = np.array(fl[trials[i]]['X'])
        a_trials.append(a)
        v_trials.append(a)
        X_trials.append(a)
    accel_subj_cond.append(a_trials)
    vel_subj_cond.append(v_trials)
    X_subj_cond.append(X_trials)
    labels_subj_cond.append(labels[k].split(';'))
marker_labels = np.array(f['marker_labels'])  # store as byte strings
f.close()

In [7]:
labels

['HC;HC01',
 'HC;HC02',
 'HC;HC03',
 'HC;HC04',
 'HC;HC05',
 'HC;HC06',
 'HC;HC07',
 'HC;HC08',
 'HC;HC09',
 'HC;HC10',
 'OFF DBS;PD01',
 'OFF DBS;PD02',
 'OFF DBS;PD03',
 'OFF DBS;PD04',
 'OFF DBS;PD05',
 'OFF DBS;PD07',
 'OFF DBS;PD08',
 'OFF DBS;PD09',
 'OFF DBS;PD10',
 'OFF DBS;PD11',
 'OFF DBS;PD12',
 'OFF DBS;PD13',
 'OFF DBS;PD14',
 'OFF DBS;PD15',
 'OFF DBS;PD16',
 'OFF DBS;PD17',
 'OFF DBS;PD18',
 'OFF DOPA;PD01',
 'OFF DOPA;PD02',
 'OFF DOPA;PD03',
 'OFF DOPA;PD05',
 'OFF DOPA;PD07',
 'OFF DOPA;PD08',
 'OFF DOPA;PD09',
 'OFF DOPA;PD10',
 'OFF DOPA;PD11',
 'OFF DOPA;PD12',
 'OFF DOPA;PD13',
 'OFF DOPA;PD14',
 'OFF DOPA;PD15',
 'OFF DOPA;PD16',
 'OFF DOPA;PD17',
 'OFF DOPA;PD18',
 'ON DBS;PD01',
 'ON DBS;PD02',
 'ON DBS;PD03',
 'ON DBS;PD04',
 'ON DBS;PD05',
 'ON DBS;PD06',
 'ON DBS;PD07',
 'ON DBS;PD08',
 'ON DBS;PD09',
 'ON DBS;PD10',
 'ON DBS;PD11',
 'ON DBS;PD12',
 'ON DBS;PD13',
 'ON DBS;PD14',
 'ON DBS;PD15',
 'ON DBS;PD16',
 'ON DBS;PD17',
 'ON DBS;PD18',
 'ON DOPA;PD01'

In [4]:
C_a_subj_cond = []
C_v_subj_cond = []
for k in range(len(accel_subj_cond)):
    C_a_trials=[]
    C_v_trials=[]
    for i in range(len(accel_subj_cond[k])):
        if ~np.any(accel_subj_cond[k][i]==0):
            print(k,i) # session/patient - trial
            C_a_trials.append(get_abs_C(accel_subj_cond[k][i]))
            C_v_trials.append(get_abs_C(vel_subj_cond[k][i]))
    C_v_subj_cond.append(C_v_trials)
    C_a_subj_cond.append(C_a_trials)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
17 0
17 1
17 2
17 3
18 0
18 1
18 2
18 3
18 4
18 5
18 6
18 7
18 8
18 9
19 0
19 1
19 2
19 3
19 4
19 5
19 6
19 7
19 8
19 9
20 0
20 1
20 2
20 3
20 4
20 5
20 6
20 7
20 8
20 9
21 0
21 1
21 2
21 3
21 4
21 5
21 6
21 7
21 8
21 9
22 0
22 1
22 2
22 3
22 4
22 5
23 0
23 1
23 2
23 3
23 4
23 5
23 

In [6]:
f = h5py.File("corr_matrices_3planes.h5",'w')
for k in range(len(accel_subj_cond)):
    fl = f.create_group(labels[k])
    C_v_=fl.create_dataset('C_v_trials',np.array(C_v_subj_cond[k]).shape)
    C_v_[...] = np.array(C_v_subj_cond[k])
    C_a_=fl.create_dataset('C_a_trials',np.array(C_a_subj_cond[k]).shape)
    C_a_[...] = np.array(C_a_subj_cond[k])
f.close()